# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
PRICE_DATA=r'C:\UFT data science program\DSI_1\scaling_to_production\05_src\data\temp\stock_px.csv'

In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the PRICE_DATA variable
price_data_path = os.getenv('PRICE_DATA')




Load the environment variable PRICE_DATA.
Use glob to find the path of all parquet files in the directory PRICE_DATA.

In [4]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\yingl\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


For each ticker and using Dask, do the following:

Add lags for variables Close and Adj_Close.

Add returns based on Adjusted Close:

returns: (Adj Close / Adj Close_lag) - 1
Add the following range:

hi_lo_range: this is the day's High minus Low.
Assign the result to dd_feat.

In [6]:
import dask.dataframe as dd

try:
    # Step 1: Read the CSV file into a Dask DataFrame
    file_path = 'C:/UFT data science program/DSI_1/scaling_to_production/05_src/data/temp/stock_px.csv'
    ddf = dd.read_csv(file_path)

    # Step 2: Add lagged variables for Close and Adj_Close
    lags = [1]  # You can add more lags if needed
    for lag in lags:
        ddf[f'Close_lag_{lag}'] = ddf['Close'].shift(lag)
        if 'Adj Close' in ddf.columns:  # Check if 'Adj Close' column exists
            ddf[f'Adj_Close_lag_{lag}'] = ddf['Adj Close'].shift(lag)
        else:
            raise KeyError("'Adj Close' column not found in DataFrame.")

    # Step 3: Add returns based on Adj_Close
    if 'Adj Close' in ddf.columns:  # Check again if 'Adj Close' column exists
        ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag_1']) - 1
    else:
        raise KeyError("'Adj Close' column not found for computing returns.")

    # Step 4: Add the hi_lo_range
    ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

    # Step 5: Assign the resulting Dask DataFrame to dd_feat
    dd_feat = ddf.compute()  # Compute to get a pandas DataFrame if needed

    print("Operation completed successfully.")

except KeyError as e:
    print(f"KeyError: {str(e)}")
    print(f"Column names in DataFrame: {ddf.columns}")

except Exception as e:
    print(f"An error occurred: {str(e)}")


KeyError: "'Adj Close' column not found in DataFrame."
Column names in DataFrame: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker', 'Close_lag_1'],
      dtype='object')


Convert the Dask data frame to a pandas data frame.
Add a rolling average return calculation with a window of 10 days.
Tip: Consider using .rolling(10).mean().

In [7]:
import dask.dataframe as dd

#  Read the CSV file into a Dask DataFrame
file_path = 'C:/UFT data science program/DSI_1/scaling_to_production/05_src/data/temp/stock_px.csv'
ddf = dd.read_csv(file_path)

# Compute the Dask DataFrame to get a pandas DataFrame
df = ddf.compute()

#  Add a rolling average return calculation with a window of 10 days
if 'Adj Close' in df.columns:
    df['rolling_avg_return'] = df['Adj Close'].pct_change().rolling(10).mean()
else:
    print("'Adj Close' column not found in pandas DataFrame.")

# Print the first few rows to verify
print(df.head())


'Adj Close' column not found in pandas DataFrame.
                        Date       Open       High        Low      Close  \
0  2013-12-02 00:00:00-05:00  27.763319  27.945016  27.349049  27.399925   
1  2013-12-03 00:00:00-05:00  27.254567  27.574352  27.189155  27.516209   
2  2013-12-04 00:00:00-05:00  27.370848  27.596153  26.963847  27.261831   
3  2013-12-05 00:00:00-05:00  27.261835  27.392656  27.014726  27.072870   
4  2013-12-06 00:00:00-05:00  27.036530  27.545281  26.992922  27.436264   

    Volume  Dividends  Stock Splits ticker  
0  2404404        0.0           0.0    JCI  
1  3774065        0.0           0.0    JCI  
2  3876632        0.0           0.0    JCI  
3  2503342        0.0           0.0    JCI  
4  2879516        0.0           0.0    JCI  


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

For calculating moving averages or returns, Dask is fully capable of handling these operations without converting to Pandas.


+ Would it have been better to do it in Dask? Why?
Yes.
Using Dask for large-scale data operations is generally better for:

Handling datasets that exceed memory capacity.
Optimizing performance through parallel processing.
Building scalable data processing pipelines.
For smaller datasets or simpler tasks, Pandas might suffice and be simpler to use. When working with large data or in a distributed environment, leveraging Dask’s capabilities can provide significant performance and memory management benefits.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.